In [ ]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import spark
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
import mysql.connector as mariadb
from mysql.connector import Error
import numpy as np
import pandas as pd
from getpass import getpass
# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Command to tell Python to actually display the graphs
%matplotlib inline

In [ ]:
user = os.getenv("user", default=None)
password = os.getenv("password", default=None)

In [ ]:
spark = SparkSession.builder.appName('req5_data_visual').getOrCreate()

In [ ]:
#creating spark dataframes, from database tables

df_loan = spark.read\
     .format("jdbc")\
     .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone")\
     .option("dbtable", "creditcard_capstone.cdw_sapp_loan_application")\
     .option("user", user)\
     .option("password", password)\
     .load()
df_loan.count()

In [ ]:
#converting spark dataframes to panda dataframes
pd_df_loan = df_loan.toPandas()

pd_df_loan


In [ ]:
pd_df_loan.isnull().sum()

5.1) Create a bar chart that shows the difference in application approvals for Married Men vs Married Women based on income ranges. (number of approvals)


In [ ]:
# Application_status = pd_df_loan['Application_Status'].value_counts()['Y']
# print(Application_status)

In [ ]:
#pd_df_cc_healthcare = pd_df_cc[pd_df_cc['TRANSACTION_TYPE'] == 'Healthcare']
pd_df_appl_status = pd_df_loan[pd_df_loan['Application_Status']=='Y']
pd_df_appl_status = pd_df_appl_status[pd_df_appl_status['Married']=='Yes']

In [ ]:
pd_df_appl_status.head()

In [ ]:
 pd_df_appl_status[pd_df_appl_status['Gender']=='Female']

In [ ]:
pd_df_appl_status.groupby(['Income','Gender'])['Application_ID'].count().to_frame().reset_index()

pd_df_appl_status.head()

In [ ]:
pd_df_appl_status_by_income_gender = pd_df_appl_status.groupby(['Income','Gender'])['Application_ID'].count().to_frame().reset_index()

In [ ]:
pd_df_appl_status_by_income_gender

In [ ]:
px.bar(pd_df_appl_status, x='Income', y='Application_ID',color = 'Gender', barmode='group')


In [ ]:
pd_df_appl_status['approved'] = 1

In [ ]:
px.bar(pd_df_appl_status, x='Income', y='approved',color = 'Gender', barmode='group')

In [ ]:
px.histogram(pd_df_appl_status, x='Income', y='approved',color = 'Gender', barmode='group', labels={'x':'Income Range', 'y':'Total Approved'})

            5.2)  Create and plot a chart that shows the difference in application approvals based on Property Area. 

In [ ]:
pd_df_property = pd_df_loan[['Application_Status','Property_Area']]

In [ ]:
pd_df_property=pd_df_property[pd_df_property['Application_Status']=='Y']

In [ ]:
Application_Status = pd_df_property.groupby(['Property_Area'])['Application_Status'].value_counts()

In [ ]:
pd_df_property

In [ ]:
Application_Status

In [ ]:
df_as = Application_Status.to_frame()

In [ ]:
df_as

In [ ]:
fig = px.histogram(pd_df_property, x='Property_Area')
fig.update_xaxes(type='category')
fig.show()

In [ ]:
Application_Status = pd_df_loan.groupby(['Property_Area'])['Application_Status'].value_counts()

In [ ]:
Application_Status

In [ ]:
fig = px.histogram(pd_df_property, x='Property_Area')
fig.update_xaxes(type='category')
fig.show()

In [ ]:
px.histogram(pd_df_loan, x='Property_Area',color = 'Application_Status', barmode='group', labels={'x':'Property_Area', 'y':'Total Applications'})

In [ ]:
pd_df_property = pd_df_loan[['Application_ID','Application_Status','Property_Area']]

In [ ]:
pd_df_property

In [ ]:
pd_as = pd_df_property.groupby(['Property_Area', 'Application_Status'])['Application_ID'].count().to_frame().reset_index()

In [ ]:
pd_as

In [ ]:
pd_as = pd_as.rename(columns={"Application_ID": "Approved"})

In [ ]:
pd_as.columns

In [ ]:
px.pie(pd_as, values='Approved', names='Property_Area', title="Difference in Approval based on Property Area" )

In [ ]:
pd_df_loan.columns

In [ ]:
pd_df_loan_approved = pd_df_loan[pd_df_loan['Application_Status'] == 'Y']

In [ ]:
groupby_columns = ['Dependents','Education','Gender','Income','Property_Area']

In [ ]:
pd_df_loan_approved.groupby(groupby_columns)['Application_ID'].count()

In [ ]:
pd_g = pd_df_loan_approved.groupby(groupby_columns)['Application_ID'].count().to_frame().reset_index()

In [ ]:
pd_s = pd_df_loan_approved.groupby(groupby_columns)['Application_ID'].count()

In [ ]:
pd_s

In [ ]:
pd_g.head(5)

In [ ]:
pd_g['Demographics'] = pd_g['Dependents'] + "_" + pd_g['Education'] + "_" + pd_g['Gender'] + \
                       "_" + pd_g['Income'] + "_" + pd_g['Property_Area']

In [ ]:
pd_g.head(5)

In [ ]:
fig = px.bar(pd_g, y='Demographics',x='Approved')
fig.update_layout(
    autosize=False,
    width=1000,
    height=1500,)

In [ ]:
spark.stop()